In [70]:
import yfinance as yf
import pandas as pd
import datetime as dt
import os

def pull_stock_data(tickers:str, start:str, end:str, interval:str, stripdateindex:str, columns:list) -> pd.DataFrame:
    """
    Returns a dataframe with chosen stock information and treasury bill returns.

    Parameters:
    -------------
    tickers (str): Yahoo finance tickers for companies divided ONLY by single space
    start (str): start date of the period (format yyyy-mm-dd)
    end (str): end date of the period (format yyyy-mm-dd)
    interval (str): wanted interval (1d, 1m, 1y)
    stripdateindex (str): argument for stripping datetime index down (d - day, m - month, y- year)
    columns (list): list of wanted values, args same as in yahoo finance
    """
    # data download
    df = yf.download(tickers = tickers, start = start, end = end, interval = interval, groupby = 'ticker')
    # changing index from datetime to just year and month
    df.index = pd.to_datetime(df.index).to_period(stripdateindex)
    # dropping na rows
    df.dropna(inplace = True)
    return df[columns]
    
def save_to_desktop(dataframe:pd.DataFrame, file_name:str):
    """
    Saves dataframe to desktop in csv format under filename provided

    Parameters:
    ------------
    dataframe (pd.DataFrame): dataframe that is to be saved
    file_name (str): name of the file
    """
    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
    dataframe.to_csv(desktop+'/'+file_name+'.csv', sep=';', encoding='UTF-8')


In [64]:
tick = "NKE MSFT XOM INTC CAT WMT JPM F UPS MKC ^IRX"
st = '2000-12-01'
en = '2020-12-31'
intv = '1mo'
strpdt = 'm'
cols = ['Adj Close']

dftest = pull_stock_data(tick, st, en, intv, strpdt, cols)
dftest.head(20)

[*********************100%***********************]  11 of 11 completed


C:\Users\maxim\AppData\Local\Temp\ipykernel_5076\161338620.py:21: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df.index = pd.to_datetime(df.index).to_period(stripdateindex)


Adj Close                                                        \
               CAT          F       INTC        JPM       MKC       MSFT   
Date                                                                       
2000-12  13.426956  12.510589  17.972366  23.754148  5.809334  13.602913   
2001-01  12.549325  15.047400  22.119835  28.748074  5.904675  19.149918   
2001-02  11.895672  15.004975  17.075617  24.571955  6.366357  18.503096   
2001-03  12.690618  15.172235  15.739316  23.645103  6.802121  17.150646   
2001-04  14.354870  15.906028  18.489393  25.267097  6.397518  21.247192   
2001-05  15.596689  13.272597  16.156532  26.096514  6.576582  21.695665   
2001-06  14.413113  13.381606  17.507397  23.611645  6.840298  22.893656   
2001-07  15.867383  13.883074  17.842579  22.990435  6.982386  20.757971   
2001-08  14.493449  10.830654  16.735287  21.076157  7.392920  17.891554   
2001-09  12.986128   9.569781  12.241850  18.267786  7.491057  16.047520   
2001-10  12.962944   8.852738  14.625544  18.915047  7.191116  18.236525   
2001-11  13.848893  10.545338  19.560616  20.375465  7.066227  20.137016   
2001-12  15.259484   8.752519  18.850256  19.635418  6.896964  20.776783   
2002-01  14.684144   8.518671  21.002003  18.393017  7.299294  19.980206   
2002-02  16.330584   8.341925  17.112083  15.949342  8.091976  18.296116   
2002-03  16.724800   9.244514  18.237642  19.439121  8.443734  18.913927   
2002-04  16.068752   8.969810  17.158142  19.139221  8.504748  16.389359   
2002-05  15.474462   9.894822  16.564415  19.793295  8.710399  15.965974   
2002-06  14.491583   9.026224  10.964852  18.675617  8.541234  17.154570   
2002-07  13.233377   7.598952  11.276931  13.742446  7.577252  15.047096   

                                                           
              NKE        UPS        WMT        XOM   ^IRX  
Date                                                       
2000-12  5.512081  33.709511  35.416008  21.869663  5.730  
2001-01  5.440067  35.516903  37.913387  21.168451  4.840  
2001-02  3.859769  32.435722  33.434532  20.388630  4.720  
2001-03  3.990307  32.753670  33.708183  20.482914  4.180  
2001-04  4.141493  33.070263  34.586430  22.404753  3.830  
2001-05  4.071165  34.221539  34.593128  22.442694  3.540  
2001-06  4.159322  33.378857  32.621159  22.197519  3.560  
2001-07  4.716827  32.818718  37.420864  21.229023  3.440  
2001-08  4.959859  31.894724  32.165890  20.410561  3.280  
2001-09  4.643424  30.120949  33.136555  20.141607  2.300  
2001-10  4.902421  29.553061  34.461662  20.167149  2.010  
2001-11  5.262952  32.577904  36.975853  19.119171  1.730  
2001-12  5.585742  31.691902  38.584995  20.208229  1.670  
2002-01  5.956800  33.424782  40.264713  20.079683  1.720  
2002-02  5.852403  34.273781  41.627460  21.236641  1.720  
2002-03  5.966742  35.474045  41.150848  22.671558  1.740  
2002-04  5.307805  35.030605  37.543331  20.778376  1.730  
2002-05  5.349606  35.228981  36.360443  20.654245  1.705  
2002-06  5.339653  36.146225  36.972050  21.288704  1.659  
2002-07  4.911561  38.247669  33.096123  19.124462  1.665

In [67]:
save_to_desktop(dftest, 'data')